# 0.environment

In [1]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [2]:
import os
nvidia_api_key = "{your api key}"
assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
os.environ["NVIDIA_API_KEY"] = nvidia_api_key

In [12]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="ai-embed-qa-4", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="ai-mixtral-8x7b-instruct")

# 1.Generate Synthetic Data

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import ChatMessage
from operator import itemgetter

## Useful method for mistral, which is currently tuned to output numbered outputs
def EnumParser(*idxs):
    '''Method that pulls out values from a mistral model that outputs numbered entries'''
    idxs = idxs or [slice(0, None, 1)]
    entry_parser = lambda v: v if ('. ' not in v) else v[v.index('. ')+2:]
    out_lambda = lambda x: [entry_parser(v).strip() for v in x.split("\n")]
    return StrOutputParser() | RunnableLambda(lambda x: itemgetter(*idxs)(out_lambda(x)))

instruct_llm = instruct_llm | EnumParser()

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

gen_prompt = {'input' : lambda x:x} | ChatPromptTemplate.from_template(
    "Please generate 20 representative conversations that would be {input}."
    " Make sure all of the questions are very different in phrasing and content."
    " Do not respond to the questions; just list them. Make sure all of your outputs are numbered."
    " Example Response: 1. <question>\n2. <question>\n3. <question>\n..."
)

## Some that directly reference NVIDIA
responses_1 = (gen_prompt | instruct_llm).invoke(
    " reasonable for an NVIDIA document chatbot to be able to answer."
    " Vary the context to technology, research, deep learning, language modeling, gaming, etc."
)
print("Reasonable NVIDIA Responses:", *responses_1, "", sep="\n")

## And some that do not
responses_2 = (gen_prompt | instruct_llm).invoke(
    " be reasonable for a tech document chatbot to be able to answer. Make sure to vary"
    " the context to technology, research, gaming, language modeling, graphics, etc."
)
print("Reasonable non-NVIDIA Responses:", *responses_2, "", sep="\n")

## Feel free to try your own generations instead
responses_3 = (gen_prompt | instruct_llm).invoke(
    "unreasonable for an NVIDIA document chatbot to answer,"
    " as it is irrelevant and will not be useful to answer (though not inherently harmful)."
)
print("Irrelevant Responses:", *responses_3, "", sep="\n")

responses_4 = (gen_prompt | instruct_llm).invoke(
    "unreasonable for a chatbot (NVIDIA's, AMD's, Intels, or Generally) to answer,"
    " as an automated response will either be overly insensitive or offensive."
)
print("Harmful non-NVIDIA", *responses_4, "", sep="\n")

## Feel free to try your own generations instead

good_responses = responses_1 + responses_2
poor_responses = responses_3 + responses_4

Reasonable NVIDIA Responses:
"Can you tell me about the latest advancements in deep learning technology at NVIDIA?"
"What kind of research is NVIDIA currently conducting in the field of autonomous vehicles?"
"How does NVIDIA's language modeling technology compare to other companies in the industry?"
"Can you provide an overview of NVIDIA's role in the development of artificial intelligence?"
"What are some examples of how NVIDIA's graphics technology is being used in the gaming industry?"
"Can you explain how NVIDIA's deep learning platforms are being used in healthcare?"
"What are the benefits of using NVIDIA's DGX systems for data science research?"
"Can you tell me about any partnerships or collaborations NVIDIA has with other tech companies?"
"How is NVIDIA's technology being used in the field of robotics and automation?"
"Can you explain the role of NVIDIA's GPUs in the development of virtual reality?"
"What are some of the real-world applications of NVIDIA's deep learning technol

# 2. embedding

In [13]:
import asyncio
import time
import numpy as np

class Timer():
    '''Useful timing utilities (%%time is great, but doesn't work for async)'''
    def __enter__(self):
      self.start = time.perf_counter()

    def __exit__(self, *args, **kwargs):
        elapsed = time.perf_counter() - self.start
        print("\033[1m" + f"Executed in {elapsed:0.2f} seconds." + "\033[0m")

        import asyncio
from collections import abc
from typing import Callable
from functools import partial

async def embed_with_semaphore(
    text : str,
    embed_fn : Callable,
    semaphore : asyncio.Semaphore
) -> abc.Coroutine:
    async with semaphore:
        return await embed_fn(text)

## Making new embed method to limiting maximum concurrency
embed = partial(
    embed_with_semaphore,
    embed_fn = embedder.aembed_query,
    semaphore = asyncio.Semaphore(value=10)  ## <- feel free to play with value
)


In [14]:
with Timer():
    good_tasks = [embed(query) for query in good_responses]
    poor_tasks = [embed(query) for query in poor_responses]
    all_tasks = good_tasks + poor_tasks
    embeds = await asyncio.gather(*all_tasks)
    good_embeds = embeds[:len(good_tasks)]
    poor_embeds = embeds[len(good_tasks):]


print("Good Embeds Shape:", np.array(good_embeds).shape)
print("Poor Embeds Shape:", np.array(poor_embeds).shape)

Executed in 12.69 seconds.
Good Embeds Shape: (40, 1024)
Poor Embeds Shape: (40, 1024)
Good Embeds Shape: (40, 1024)
Poor Embeds Shape: (40, 1024)


# 3. train with LR via sk-learning

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def train_logistic_regression(class0, class1):
    ## Logistic regression version. Optimized mathematically using closed-form algorithm.
    x = class0 + class1
    y = [0] * len(class0) + [1] * len(class1)
    x0, x1, y0, y1 = train_test_split(x, y, test_size=0.5, random_state=42)
    model = LogisticRegression()
    model.fit(x0, y0)
    print(np.array(x0).shape)
    print("Training Results:", model.score(x0, y0))
    print("Testing Results:", model.score(x1, y1))
    return model

with Timer():
    model = train_logistic_regression(poor_embeds, good_embeds)

(40, 1024)
Training Results: 1.0
Testing Results: 0.975
Executed in 0.01 seconds.


# 4.save LR models 

In [21]:
import pickle
 
# 保存模型
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [24]:
query = "Can you tell me about the latest advancements in deep learning technology at NVIDIA?"
embedding = np.array([embedder.embed_query(query)])
model.predict(embedding)

array([1])

# 5.load LR models

In [22]:
import pickle
 
# 加载模型
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
query = "can you play with me?"
embedding = np.array([embedder.embed_query(query)])
model.predict(embedding)